In [43]:
# import pandas as pd

# # Mengambil data dari file CSV
# df = pd.read_csv('male_players.csv')   

# # Mengambil data dari kolom fifa_version 22
# df = df[df['fifa_version'] > 21]

# df.head()

# # menghapus kolom yang kosong dan duplicate
# df = df.dropna(axis=1)
# df = df.drop_duplicates()

# # Menyimpan data ke file CSV
# df.to_csv('data_fifa.csv', index=False)


***DATA UNDERSTANDING***

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
import warnings
import joblib
warnings.simplefilter("ignore", UserWarning)

# Memuat dataset dari file Excel
url = 'https://raw.githubusercontent.com/Khalis189/Classification_fifa24/refs/heads/main/data_fifa.csv'
df = pd.read_csv(url)

# Menampilkan beberapa baris pertama dari dataframe untuk memahami strukturnya
display(df.head())

,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,/player/158023/lionel-messi/240002,24.0,2.0,2023-09-22,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90,...,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,/player/165153/karim-benzema/240002,24.0,2.0,2023-09-22,K. Benzema,Karim Benzema,"CF, ST",90,90,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3


***DATA PREPROCESSING***

In [45]:
# Hanya menggunakan data dengan kolom fifa_version = 24.0
df = df[df['fifa_version'] == 24]

# Memilih kolom yang relevan untuk pemodelan
kolom_dipakai = ['age', 'height_cm', 'weight_kg',
                 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve',
                 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
                 'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping',
                 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression',
                 'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness',
                 'defending_standing_tackle', 'defending_sliding_tackle']

# Menghapus baris dengan nilai NaN (jika ada)
df = df[kolom_dipakai + ['player_positions']].dropna()

# Menghapus data duplicate (jika ada)
df = df.drop_duplicates()

# Mengidentifikasi semua posisi unik dalam kolom 'player_positions'
posisi_unik = ['ST', 'LW', 'RW', 'CM', 'CB', 'LB', 'RB', 'GK']

***EDA***

In [46]:
# # Melihat jumlah data kolom 'player_positions' untuk mengecek keseimbangan kelas
# df['player_positions'].value_counts()

# # Melakukan teknik under-sampling untuk membuat kelas yang seimbang dengan randomundersampler
# rus = RandomUnderSampler(random_state=101)
# X_res, y_res = rus.fit_resample(df.drop(columns=['player_positions']), df['player_positions'])
# df_res = pd.concat([X_res, y_res], axis=1)

# # Melihat jumlah data kolom 'player_positions' setelah dilakukan under-sampling
# df_res['player_positions'].value_counts()


***PEMODELAN***

In [47]:
# Definisikan parameter grid untuk GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Melatih model untuk setiap posisi unik dengan hyperparameter tuning
models = {}
scalers = {}

# Menghitug rata-rata akurasi dari setiap model
data = []

for posisi in posisi_unik:
    print(f"Melatih model untuk posisi: {posisi}")
    
    # Mengubah kolom target menjadi 1 untuk pemain posisi terpilih dan 0 untuk pemain non-posisi terpilih
    df['target'] = df['player_positions'].apply(lambda x: 1 if posisi in x.split(',') else 0)
    
    # Melakukan teknik under-sampling untuk membuat kelas yang seimbang dengan RandomUnderSampler
    rus = RandomUnderSampler(random_state=101)
    X_res, y_res = rus.fit_resample(df.drop(columns=['player_positions', 'target']), df['target'])
    df_res = pd.concat([X_res, y_res], axis=1)
    
    # Membagi dataset menjadi atribut dan target
    X = df_res.drop(columns=['target'])
    y = df_res['target']
    
    # Membagi data menjadi set pelatihan dan pengujian
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standarisasi fitur
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Membuat model RandomForestClassifier
    model = RandomForestClassifier()
    
    # Inisialisasi GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
    
    # Melatih model dengan GridSearchCV
    grid_search.fit(X_train_scaled, y_train)
    
    # Menampilkan parameter terbaik
    print(f"Best parameters for {posisi}: {grid_search.best_params_}")
    
    # Memprediksi dengan model terbaik
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_scaled)
    
    # Menghitung metrik evaluasi
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    data.append(accuracy_score(y_test, y_pred))
    
    # print("Confusion Matrix:")
    # plt.figure(figsize=(15, 10))
    # sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='coolwarm')
    # plt.title(f'Confusion Matrix for {posisi}')
    # plt.xlabel('Predicted')
    # plt.ylabel('Actual')
    # plt.show()
    
    # Menyimpan model dan scaler ke file pickle
    joblib.dump(best_model, f'model_{posisi}.pkl')
    joblib.dump(scaler, f'scaler_{posisi}.pkl')

print("Model dan scaler telah disimpan ke file pickle.")

# # Fungsi untuk memprediksi posisi terbaik pemain berdasarkan input user
# def prediksi_posisi_pemain(input_data):
#     prediksi = {}
#     for posisi, model in models.items():
#         scaler = scalers[posisi]
#         input_scaled = scaler.transform([input_data])
#         prediksi[posisi] = model.predict(input_scaled)[0]
#     posisi_terbaik = max(prediksi, key=prediksi.get)
#     return posisi_terbaik

Melatih model untuk posisi: ST
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters for ST: {'bootstrap': False, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       500
           1       0.95      0.96      0.95       483

    accuracy                           0.96       983
   macro avg       0.96      0.96      0.96       983
weighted avg       0.96      0.96      0.96       983

Melatih model untuk posisi: LW
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters for LW: {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.79      0.84        75
           1       0.84      0.92      0.88        92

    accuracy  

In [48]:
# # Fungsi untuk memprediksi posisi terbaik pemain berdasarkan input user
# def prediksi_posisi_pemain(input_data):
#     prediksi = {}
#     for posisi, model in models.items():
#         scaler = scalers[posisi]
#         input_scaled = scaler.transform([input_data])
#         prediksi[posisi] = model.predict(input_scaled)[0]
#     posisi_terbaik = max(prediksi, key=prediksi.get)
#     return posisi_terbaik

# # Contoh penggunaan fungsi prediksi
# input_user = [25, 180, 75, 70, 85, 60, 80, 75, 85, 70, 65, 80, 85,
#               90, 85, 80, 85, 80, 75, 70, 80, 85, 75, 70, 65,
#               60, 70, 75, 80, 85, 70, 75]
# posisi_terbaik = prediksi_posisi_pemain(input_user)
# print(f"Posisi terbaik untuk pemain ini adalah: {posisi_terbaik}")

***RATA-RATA AKURASI SETIAP MODEL***

In [49]:
# Menghitung rata-rata akurasi setiap model
value = np.array(data)

display(np.mean(value))

np.float64(0.907455757388695)